In [1]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm
from sklearn import linear_model
from scipy.io import arff
import pandas as pd
import gzip
from collections import defaultdict 

def parseDataFromURL(fname):
    for l in urlopen(fname):
        yield eval(l)

def parseData(fname):
    for l in open(fname):
        yield eval(l)


f = open('question_tags.csv', encoding="utf8")
header = f.readline()
header = header.strip().split(',')
dataset = []

In [2]:
header = f.readline() # id - tag

In [3]:
n = 0
for line in f:
    fields = line.strip().split(',')
    d = list(fields)
    dataset.append(d)
    n += 1
    if n>=100000: # treat 10,0000 lines each time in case the kernel dies again!
        break

In [4]:
dataset1 = pd.DataFrame(dataset)

In [5]:
g1 = dataset1.groupby(0) # group the first 100,0000 samples by question id

In [6]:
group_names = []
for name,group in g1: 
    group_names.append(name)

In [7]:
len(    group_names)

35661

In [7]:
import numpy as np
group_names = np.array(group_names, dtype=object)

In [10]:
(group_names)

array(['10000', '100001', '100003', ..., '99995', '99996', '99999'],
      dtype=object)

In [12]:
### for each question, note the tags that show up together
import itertools as it

In [13]:
tag_net = {} # key: a tuple of two tags; value: times they show up together in one question
all_tags = []
for num in group_names:
    question = g1.get_group(num)
    tags = list(question[1])
    for tag in tags:
        if tag not in all_tags:
            all_tags.append(tag)
    combi = list(it.combinations(tags, 2)) # combination of the tags
    for c in combi:
        if c in tag_net:
            tag_net[c] += 1
        else:
            tag_net[c] = 1

In [14]:
tag_net

{('python', 'oop'): 13,
 ('python', 'metaclass'): 3,
 ('python', 'python-datamodel'): 4,
 ('oop', 'metaclass'): 1,
 ('oop', 'python-datamodel'): 1,
 ('metaclass', 'python-datamodel'): 1,
 ('c#', 'logging'): 15,
 ('c#', 'enterprise-library'): 4,
 ('c#', 'application-blocks'): 1,
 ('logging', 'enterprise-library'): 6,
 ('logging', 'application-blocks'): 1,
 ('enterprise-library', 'application-blocks'): 2,
 ('malware', 'bho'): 1,
 ('malware', 'virus'): 1,
 ('malware', 'browser'): 1,
 ('bho', 'virus'): 1,
 ('bho', 'browser'): 1,
 ('virus', 'browser'): 1,
 ('php', 'facebook'): 3,
 ('php', 'rss'): 7,
 ('facebook', 'rss'): 1,
 ('c#', 'regex'): 36,
 ('algorithm', 'sorting'): 20,
 ('algorithm', 'lisp'): 1,
 ('algorithm', 'list'): 6,
 ('sorting', 'lisp'): 1,
 ('sorting', 'list'): 4,
 ('lisp', 'list'): 3,
 ('wcf', 'exception'): 8,
 ('wcf', 'optimization'): 1,
 ('exception', 'optimization'): 2,
 ('testing', 'browser'): 3,
 ('testing', 'user-interface'): 2,
 ('browser', 'user-interface'): 2,
 ('wcf

In [15]:
len(tag_net) # all edges and the weight

58155

In [16]:
(all_tags) # all nodes

['stackoverflow',
 'web-frameworks',
 'python',
 'oop',
 'metaclass',
 'python-datamodel',
 'c#',
 'logging',
 'enterprise-library',
 'application-blocks',
 'malware',
 'bho',
 'virus',
 'browser',
 'php',
 'facebook',
 'rss',
 'regex',
 'algorithm',
 'sorting',
 'lisp',
 'list',
 'wcf',
 'exception',
 'optimization',
 'testing',
 'user-interface',
 'silverlight',
 'compression',
 'gzip',
 'linq-to-sql',
 'unit-testing',
 'resharper',
 'windows-server-2003',
 'winsock',
 'design-patterns',
 'osx',
 'text-editor',
 'java',
 'content-management-system',
 '.net',
 'mysql',
 'data-binding',
 'perl',
 'obfuscation',
 'puzzle',
 'deployment',
 'verifyerror',
 'swing',
 'shortcut',
 'keystroke',
 'c++',
 'c',
 'indexing',
 'code-search-engine',
 'tdd',
 'companies',
 'seattle',
 'closed',
 'education',
 'hardware',
 'cpu',
 'motherboard',
 'psu',
 'eclipse',
 'eclipse-3.4',
 'ganymede',
 'version-control',
 'tools',
 'software-tools',
 'free',
 'firewall',
 'sygate',
 'needs-to-be-deleted',
 

In [17]:
# write a file to save the nodes:
import csv
tag_list = []
for i in range (len(all_tags)):
    l = [all_tags[i]]
    tag_list.append(l)
with open('nodes.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(tag_list)
csvFile.close()

In [19]:
len(tag_list)

8357

In [68]:
edge_list = []
for net in tag_net:
    source, target = net
    weight = tag_net[net]
    edge_list.append([source, target, weight])

In [69]:
# write all edges:
with open('edges.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(edge_list)
csvFile.close()